Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,Symbol[],[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.036906327245684434, θ=0.004944461009804169)  …  Distributions.Gamma(α=0.8899522221277354, θ=0.6125149461844521)  
 Distributions.Gamma(α=0.9769062513668332, θ=0.35546258180306567)        Distributions.Gamma(α=0.4976589867146315, θ=0.20255076620072643) 
 Distributions.Gamma(α=0.5497918359127176, θ=0.8444848002124252)         Distributions.Gamma(α=0.9638684417975971, θ=0.47762321004403874) 
 Distributions.Gamma(α=0.5608996960041583, θ=0.9249522113867419)         Distributions.Gamma(α=0.16431939223729408, θ=0.13050064531274175)
 Distributions.Gamma(α=0.12174900070191641, θ=0.7009982579450706)        Distributions.Gamma(α=0.9500158917005916, θ=0.3561428828858668)  )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.461359    0.570902  0.346279  0.667669
 0.00037266  0.875459  0.641215  0.465467
 0.204771    0.463705  0.261236  0.460177)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7761"),Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.665157  0.710955  0.301238  0.86269 
 0.887504  0.718267  0.472681  0.339745
 0.472096  0.29194   0.04784   0.188863)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,5,3],Dict(:a=>[1,2],:b=>[2,3]),Dict(:a=>[1,2],:b=>[1,2]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,Symbol[],[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.2554197765863395, σ=0.8754758603238213)     …  Distributions.Normal(μ=0.1533659161245713, σ=0.2360716831183305) 
 Distributions.Normal(μ=0.23547407364073947, σ=0.028540367516222798)     Distributions.Normal(μ=0.7777150230960888, σ=0.8986755525826409) 
 Distributions.Normal(μ=0.1821523988053031, σ=0.6161215816966299)        Distributions.Normal(μ=0.8397666570459339, σ=0.5805456181529169) 
 Distributions.Normal(μ=0.18951653196626905, σ=0.8499119409753242)       Distributions.Normal(μ=0.5281465333420308, σ=0.64355464626486)   
 Distributions.Normal(μ=0.39806271488788303, σ=0.9046435142533806)       Distributions.Normal(μ=0.5025579222977241, σ=0.17738138931339686))

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.60558   0.633834  0.763185  0.598406 
 0.771013  0.724959  0.727411  0.0277171
 0.541688  0.220069  0.274855  0.643276 )

Make a multivariate node

In [8]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]
d[i, j] ~ MvNormal(m, VV)
d.innerinds, d.outerinds

([:i],[:j])

# Make a factor

In [9]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;
value(f)

-33.92396820826474

and a multivariate factor...

In [10]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]

y[i, j] ~ MvNormal(m, VV)
μ[i] ~ Normal(zeros(dims[1]), ones(dims[1]))
Σ[j] ~ Const([diagm(rand(dims[1])) for x in 1:dims[2]])

f = @factor LogMvNormalCanonFactor y μ Σ
value(f)

-28.08487173898473

In [11]:
vars = fieldnames(f)
val_expr = value(typeof(f))
using Base.Cartesian
macroexpand(:(@wrapvars $vars $val_expr (@ntuple 1 i)))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 373:
    δ = E(project(f,:x,(i_1,))) - E(project(f,:μ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 374:
    EL = E(project(f,:Λ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 375:
    EΛ = if ndims(EL) == 1
            diagm(EL)
        else 
            EL
        end # /Users/jmxp/code/vbgraph/types.jl, line 376:
    -(1 / 2) * ((trace(EΛ * ((V(project(f,:x,(i_1,))) .+ V(project(f,:μ,(i_1,)))) .+ δ * δ')) + length(project(f,:x,(i_1,))) * log(2π)) - Elogdet(project(f,:Λ,(i_1,))))
end

In [12]:
f.inds.indices, f.inds.inds_in_factor, f.inds.inds_in_node

([:j],Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]),Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]))

In [13]:
project_inds(f, :x, (5,)), project_inds(f, :μ, (5,))

(Any[5],Any[Colon()])

In [14]:
μ

VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:i],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)])

In [15]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [16]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [17]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1]),Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [18]:
value(obs), value(μ_prior), value(τ_prior)

(-173.70865093705677,-13.593072740907873,0.0449816833123928)

In [19]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,Symbol[],[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,Symbol[],[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,Symbol[],[:i,:j],20x6 Array{Float64,2}:
 0.768338  0.103625  0.510894    0.495383  0.948134   0.615042 
 0.239501  0.957119  0.382016    0.144051  0

In [20]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [21]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [22]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…

In [23]:
naturals(obs, μ)

Distributions.Normal
Array{Tuple{Float64,Float64},1}
[(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.8451718129005884,-0.55)
Any[1]
(0.0,0.0)
Array{Tuple{Float64,Float64},1}
[(0.8451718129005884,-0.55),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.26345159565393184,-0.55)
Any[1]
(0.8451718129005884,-0.55)
Array{Tuple{Float64,Float64},1}
[(1.1086234085545201,-1.1),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.2267795457177972,-0.55)
Any[1]
(1.1086234085545201,-1.1)
Array{Tuple{Float64,Float64},1}
[(1.3354029542723174,-1.6500000000000001),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5070858550482072,-0.55)
Any[1]
(1.3354029542723174,-1.6500000000000001)
Array{Tuple{Float64,Float64},1}
[(1.8424888093205247,-2.2),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.668523188229438,-0.55)
Any[1]
(1.8424888093205247,-2.2)
Array{Tuple{Fl

6-element Array{Tuple{Float64,Float64},1}:
 (9.516477897358065,-11.000000000000002) 
 (12.042748536310267,-11.000000000000002)
 (10.565344873618535,-11.000000000000002)
 (12.718486025465547,-11.000000000000002)
 (13.858597441469364,-11.000000000000002)
 (10.211690493873178,-11.000000000000002)

In [24]:
naturals(obs, τ)

Distributions.Gamma
Array{Tuple{Float64,Float64},1}
[(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5,0.7951716501329202)
Any[1]
(0.0,0.0)
Array{Tuple{Float64,Float64},1}
[(0.5,0.7951716501329202),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5,0.5286804724184309)
Any[1]
(0.5,0.7951716501329202)
Array{Tuple{Float64,Float64},1}
[(1.0,1.323852122551351),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5,0.5212516373371778)
Any[1]
(1.0,1.323852122551351)
Array{Tuple{Float64,Float64},1}
[(1.5,1.845103759888529),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5,0.6062545720619716)
Any[1]
(1.5,1.845103759888529)
Array{Tuple{Float64,Float64},1}
[(2.0,2.451358331950501),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5,0.6846790302481209)
Any[1]
(2.0,2.451358331950501)
Array{Tuple{Float64,Float64},1}
[(2.5,3.1360373621986217),(0.0,0.0),(0.

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,12.389455657426442)
 (10.0,13.874953072967584)
 (10.0,12.897290937210943)
 (10.0,13.973109415076216)
 (10.0,14.751745547384866)
 (10.0,12.896466621898094)

In [25]:
check_conjugate(τ, m)

true

In [26]:
check_conjugate(μ, m)

true

In [27]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [28]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [29]:
update!(m)

Distributions.Normal
Array{Tuple{Float64,Float64},1}
[(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.8451718129005884,-0.55)
Any[1]
(0.0,0.0)
Array{Tuple{Float64,Float64},1}
[(0.8451718129005884,-0.55),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.26345159565393184,-0.55)
Any[1]
(0.8451718129005884,-0.55)
Array{Tuple{Float64,Float64},1}
[(1.1086234085545201,-1.1),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.2267795457177972,-0.55)
Any[1]
(1.1086234085545201,-1.1)
Array{Tuple{Float64,Float64},1}
[(1.3354029542723174,-1.6500000000000001),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.5070858550482072,-0.55)
Any[1]
(1.3354029542723174,-1.6500000000000001)
Array{Tuple{Float64,Float64},1}
[(1.8424888093205247,-2.2),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0),(0.0,0.0)]
Tuple{Float64,Float64}
(0.668523188229438,-0.55)
Any[1]
(1.8424888093205247,-2.2)
Array{Tuple{Fl

In [30]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-1.0490085388182389, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.7373345610973354, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.30734758734183076, σ=0.7071067811865476)
 Distributions.Normal(μ=1.2151531219810048, σ=0.7071067811865476)  
 Distributions.Normal(μ=2.021333635545501, σ=0.7071067811865476)   
 Distributions.Normal(μ=-0.5574189974560955, σ=0.7071067811865476) 

In [31]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=38.46897805147462, θ=0.8999999999999999) 
 Distributions.Gamma(α=17.23927134428458, θ=0.8999999999999999) 
 Distributions.Gamma(α=22.793946564696697, θ=0.8999999999999999)
 Distributions.Gamma(α=20.689164149390557, θ=0.8999999999999999)
 Distributions.Gamma(α=36.1434157093273, θ=0.8999999999999999)  
 Distributions.Gamma(α=27.178344443171977, θ=0.8999999999999999)

# A Multivariate Model

We want a simple model of inference for a multivariate normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\pmb{\mu}, \Sigma) \\
    \mu_i \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

In [32]:
d = 5
μ[i] ~ MvNormalCanon(zeros(d), diagm(ones(d)))
Λ[i, i] ~ Wishart(float(d), diagm(ones(d)))
x[i, j] ~ Const(randn(d, 20))
f = @factor LogMvNormalCanonFactor x μ Λ

VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
  1.40541   0.676342     0.522476  …  -0.0735204  -0.522661  -0.316096
 -0.321352  1.12056     -0.484001     -0.368478   -0.779518  -0.810697
  2.03589   0.380064     1.94204       0.0392948  -0.322304   1.07383 
  0.692768  0.00983499  -0.342216     -2.03201     3.47206   -1.51525 
  0.437605  0.252794     0.809141      1.45973     0.763536  -0.435839),VB.RandomNode{Distributions.MvNormalCanon{P<:PDMats.AbstractPDMat,V<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormalCanon[FullNormalCanon(
μ: [0.0,0.0,0.0,0.0,0.0]
h: [0.0,0.0,0.0,0.0,0.0]
J: PDMats.PDMat(5,5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0,5x5 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.

In [33]:
naturals(f, μ)

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
Distributions.MvNormalCanon{P<:PDMats.AbstractPDMat,V<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}
Array{Tuple{Array{Float64,1},Array{Float64,2}},1}
[([0.0,0.0,0.0,0.0,0.0],
[0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0])]
Tuple{Array{Float64,1},Array{Float64,2}}
([7.027069080016134,-1.6067584918986755,10.17943706389522,3.463838992900033,2.188026327378401],
[-2.5 -0.0 -0.0 -0.0 -0.0
 -0.0 -2.5 -0.0 -0.0 -0.0
 -0.0 -0.0 -2.5 -0.0 -0.0
 -0.0 -0.0 -0.0 -2.5 -0.0
 -0.0 -0.0 -0.0 -0.0 -2.5])
Any[1]
([0.0,0.0,0.0,0.0,0.0],
[0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0])
Array{Tuple{Array{Float64,1},Array{Float64,2}},1}
[([7.027069080016134,-1.6067584918986755,10.17943706389522,3.463838992900033,2.188026327378401],
[-2.5 0.0 0.0 0.0 0.0
 0.0 -2.5 0.0 0.0 0.0
 0.0 0.0 -2.5 0.0 0.0
 0.0 0.0 0.0 -2.5 0.0
 

1-element Array{Tuple{Array{Float64,1},Array{Float64,2}},1}:
 ([10.310291464973611,-49.88861591617785,41.95570663412767,-15.36841703515713,8.390985823486933],
5x5 Array{Float64,2}:
 -50.0    0.0    0.0    0.0    0.0
   0.0  -50.0    0.0    0.0    0.0
   0.0    0.0  -50.0    0.0    0.0
   0.0    0.0    0.0  -50.0    0.0
   0.0    0.0    0.0    0.0  -50.0)

In [34]:
naturals(f, Λ)

Distributions.Wishart{ST<:PDMats.AbstractPDMat}
Array{Tuple{Array{Float64,2},Float64},1}
[(
[0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0],

0.0)]
Tuple{Array{Float64,2},Int64}
(
[1.4875939971063759 -0.22581605834949073 1.4306321488733664 0.4868127177032409 0.30750824302764046
 -0.22581605834949073 0.5516334570257702 -0.3271179389032353 -0.11131105432823768 -0.07031259764026235
 1.4306321488733664 -0.3271179389032353 2.572418778756075 0.7051986205538417 0.4454575258738847
 0.4868127177032409 -0.11131105432823768 0.7051986205538417 0.7399636113746944 0.15157941820530318
 0.30750824302764046 -0.07031259764026235 0.4454575258738847 0.15157941820530318 0.5957491841860203],

0)
Any[1]
(
[0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0],

0.0)
Array{Tuple{Array{Float64,2},Float64},1}
[(
[1.4875939971063759 -0.22581605834949073 1.4306321488733664 0.4868127177032409 0.307508243027

1-element Array{Tuple{Array{Float64,2},Float64},1}:
 (
5x5 Array{Float64,2}:
 14.8795    -2.85753   3.33315  -1.17461   0.790309
 -2.85753   20.9334   -3.7247    2.26937   2.44113 
  3.33315   -3.7247   24.7086   -9.60959  -2.99314 
 -1.17461    2.26937  -9.60959  29.7526    3.30383 
  0.790309   2.44113  -2.99314   3.30383  20.6381  ,

0.0)

In [35]:
d = 5
μ[i] ~ MvNormal(zeros(d), diagm(ones(d)))
        τ[i] ~ Gamma(1.1 * ones(d), ones(d))
        x[i, j] ~ Const(randn(d, 20))
        f = @factor LogMvNormalCanonFactor x μ τ


VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
 -0.49872   -0.592252  -0.10265    …  -0.905863  -0.631664  -2.36712 
  1.15443   -1.11883    1.45364        2.86237   -0.999158  -0.818676
  0.353471  -0.948522   0.0364618      0.234219  -1.76751   -0.106651
 -1.16931    1.08477   -0.936911      -1.5945     0.356439  -1.02448 
 -0.260462  -0.348656  -0.337611      -1.3613    -0.135999   1.84388 ),VB.RandomNode{Distributions.MvNormal{Cov<:PDMats.AbstractPDMat,Mean<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormal[FullNormal(
dim: 5
μ: [0.0,0.0,0.0,0.0,0.0]
Σ: 5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0
)
]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:i],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ

In [36]:
naturals(f, τ)

Distributions.Gamma
Array{Tuple{Float64,Float64},0}
(0.0,0.0)
Tuple{Float64,Float64}
(2.5,6.012079523975978)
Any[Colon()]
[(0.0,0.0)]


LoadError: LoadError: MethodError: `.+` has no method matching .+(::Tuple{Float64,Float64}, ::Float64)
Closest candidates are:
  .+(!Matched::Range{T}, ::Real)
  .+(!Matched::Range{T}, ::Number)
  .+(!Matched::Number, ::Number)
  ...
while loading In[36], in expression starting on line 1